# PC2 - Grupo 2 (Python)

#### Integrantes
- GARCIA RODRIGUEZ, EMILIO ALONSO
- PADILLA AQUISE, ALESSANDRO PIERO
- RIEGA NUÑEZ, GABRIEL ANTONIO FERMIN
- SALAMANCA FERNANDEZ, LUCAS PABLO
- SILVA ANDUJAR, NICOLAS

#### 1. Loading and processing the data


In [121]:
import numpy as np
import pandas as pd

In [122]:
data= "https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv"

df =pd.read_csv(data)

In [123]:
df = df.set_index('rownames')

#### As in Group Assignment 1 2024 - 2 #1044 , generate the extra-flexible model. This means that it contains all two-way interactions between the experience polynomials and the indicator variables

In [124]:
basic_predictors = ["sex", "hsg", "scl", "clg", "ad", "so", "we", "ne", "exp1", "occ2", "ind2"]
df_with_dummies = pd.get_dummies(df[basic_predictors], columns = ["occ2", "ind2"], drop_first = True, dtype = int) 
X_extra_flexible = df_with_dummies.copy()
X_extra_flexible = pd.concat((X_extra_flexible, df[["exp2", "exp3", "exp4"]]), axis = 1)
extra_flexible_base_features = X_extra_flexible.columns[1:]


In [125]:
base_experience_variables = ["exp1", "exp2", "exp3"]
exp_4 = ["exp4"]
education_variables = ["hsg", "scl", "clg", "ad"]
location_variables = ["so", "we", "ne"]
occupation_variables = [variable for variable in X_extra_flexible.columns if "occ2" in variable]
industry_variables = [variable for variable in X_extra_flexible.columns if "ind2" in variable]
variable_groups = [base_experience_variables, exp_4, education_variables, location_variables, occupation_variables, industry_variables]

In [126]:
for variable_group_index, variable_group in enumerate(variable_groups, 1):
        if variable_group == len(variable_groups):
                break

        other_variable_groups = variable_groups[variable_group_index:]

        for variable in variable_group:
                for other_variables in other_variable_groups:
                        temp = X_extra_flexible[other_variables].multiply(X_extra_flexible[variable], axis = "index")
                        temp = temp.add_prefix(f"{variable}")
                        X_extra_flexible = pd.concat(
                                (X_extra_flexible, temp),
                                axis = 1
                        )

In [127]:
temp = pd.DataFrame({"exp8": X_extra_flexible["exp4"] ** 2})
extra_flexible_model_array = pd.concat((X_extra_flexible, temp), axis = 1)

In [128]:
print(extra_flexible_model_array.shape)

(5150, 968)


#### 2.1. Generate the array for the outcome variable Y and normalize it

In [129]:
df_logwage = df[['lwage']]
log_w = np.array(df_logwage['lwage'])
log_w = log_w.reshape(-1, 1)

In [130]:
norm_log_w = (log_w - np.mean(log_w)) / np.std(log_w)

print(norm_log_w)

[[-1.24037498]
 [ 1.5815695 ]
 [-0.99532021]
 ...
 [ 1.19031569]
 [ 0.9200321 ]
 [-0.20976589]]


In [131]:
continuous_columns = base_experience_variables + exp_4

In [132]:
for var in continuous_columns:
    if var in extra_flexible_model_array.columns:
        mean = extra_flexible_model_array[var].mean()
        std = extra_flexible_model_array[var].std()
        extra_flexible_model_array[var] = (extra_flexible_model_array[var] - mean) / std

# Verifica que se aplicó la normalización correctamente
print(extra_flexible_model_array[continuous_columns])

              exp1      exp2      exp3      exp4
rownames                                        
10       -0.637222 -0.632088 -0.544750 -0.464746
12        1.624909  1.647396  1.487693  1.256002
15        0.399588  0.055256 -0.165910 -0.273125
18        1.059376  0.807586  0.509984  0.260497
19        0.776610  0.455166  0.166481 -0.031616
...            ...       ...       ...       ...
32620    -0.448711 -0.552106 -0.518109 -0.456974
32624    -0.165945 -0.394642 -0.449160 -0.430494
32626    -0.260200 -0.452129 -0.476561 -0.441880
32631    -0.354455 -0.504617 -0.499405 -0.450550
32643     0.022566 -0.264671 -0.379038 -0.397466

[5150 rows x 4 columns]


#### Split between training and testing samples. The testing sample should be 10% of the total.

In [133]:
from sklearn.model_selection import train_test_split
ef_model =extra_flexible_model_array
y=norm_log_w

In [134]:
ef_train,ef_test,y_ef_train,y_ef_test = train_test_split(ef_model,y, train_size= 0.1)


### 3. Creating the Lasso Cross - Validation Procedure 

4. Program a function that generates a logarithmically spaced grid. The input arguments should be the lower and upper bounds of the grid, as well as the natural logarithm of the spacing between each element of the grid. The output should be the logarithmically spaced grid, meaning that if we take the natural logarithm of each entry in the grid, they will be equally spaced. This will be the grid of values for λ values to try during cross-validation.

In [135]:

def log_spaced_grid(log_lower_bound, log_upper_bound, log_spacing, num_points):
    log_grid = np.linspace(log_lower_bound, log_upper_bound, num_points)
    return np.exp(log_grid)


5. Program a function to generate 
k
 folds. It should take as input the array to be split rowwise and the number of folds desired. It should output a list of 
k
 1d arrays of booleans; these arrays should all be the same length as the number of rows in the input array, and when they are all summed together they should add up to an array of all true values. Create your own procedure for splitting. You can aid yourself with third party packages like numpy in Python or Statistics in Julia, but do not use a pre-programmed third party splitting procedure like sk-learns's KFolds in Python.

In [136]:
def generate_k_folds(X, k):
    n_samples = X.shape[0]
    fold_sizes = np.full(k, n_samples // k, dtype=int)
    fold_sizes[:n_samples % k] += 1  # Distribute remaining samples
    
    # Create an array of indices and shuffle it
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    # Create the folds
    folds = []
    current = 0
    for fold_size in fold_sizes:
        start, stop = current, current + fold_size
        mask = np.zeros(n_samples, dtype=bool)
        mask[indices[start:stop]] = True
        folds.append(mask)
        current = stop
    
    return folds

6. Program a function that integrates those that you programmed in the last two items to find the value of 
λ
 that minimizes the testing mean square error across folds.

In [137]:
from sklearn.linear_model import Lasso

import numpy as np
from sklearn.linear_model import Lasso

def find_optimal_lambda(Y, X, lambda_bounds, k):
    log_lower = np.log(lambda_bounds[0])
    log_upper = np.log(lambda_bounds[1])
    num_points = int((log_upper - log_lower) / 0.1) + 1  
    
    lambdas = np.exp(np.linspace(log_lower, log_upper, num_points))
    
    folds = generate_k_folds(X, k)
    
    all_mse = np.zeros((len(lambdas), k))
    
    for i, lambda_val in enumerate(lambdas):
        for j, fold in enumerate(folds):
            X_train, X_test = X[~fold], X[fold]
            Y_train, Y_test = Y[~fold], Y[fold]
            
            model = Lasso(alpha=lambda_val, fit_intercept=True)
            model.fit(X_train, Y_train)
            Y_pred = model.predict(X_test)
            
            all_mse[i, j] = np.mean((Y_test - Y_pred)**2)

    avg_mse = np.mean(all_mse, axis=1)
    optimal_index = np.argmin(avg_mse)
    optimal_lambda = lambdas[optimal_index]
    
    optimal_model = Lasso(alpha=optimal_lambda, fit_intercept=True)
    optimal_model.fit(X, Y)
    
    result = {
        'optimal_lambda': optimal_lambda,
        'optimal_coef': optimal_model.coef_,
        'all_lambdas': lambdas,
        'all_mse': avg_mse
    }
    
    return result


7. Program a function for predicting the outcome variable through model estimated with the optimal lambda. It should take as inputs

In [138]:

def lasso_predict(optimal_model, X):
    optimal_lambda = optimal_model['optimal_lambda']
    optimal_coef = optimal_model['optimal_coef']

    model = Lasso(alpha=optimal_lambda, fit_intercept=True)
    model.coef_ = optimal_coef
    model.intercept_ = 0  

    predictions = model.predict(X)
    
    return predictions

# 3.Applying the Lasso Cross-Validation Procedure

8. Fit a simple OLS model with the training sample.

In [139]:
import statsmodels.api as sm

X_train = sm.add_constant(ef_train)

# Create and fit the OLS model
model = sm.OLS(y_ef_train, X_train)
results = model.fit()

# Print the summary of the model
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.937
Model:                            OLS   Adj. R-squared:                  0.365
Method:                 Least Squares   F-statistic:                     1.638
Date:                Sun, 22 Sep 2024   Prob (F-statistic):             0.0155
Time:                        01:01:15   Log-Likelihood:                -30.549
No. Observations:                 515   AIC:                             989.1
Df Residuals:                      51   BIC:                             2958.
Df Model:                         463                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            -27.0990     18.427     -1.

9. Find the optimal lambda and its related coefficients with the function programmed in 3. and the training sample. The natural logarithm of the numbers in the grid of $\lambda$s should be spaced by 0.1, and the natural logarithm of the lower and upper bounds of the grid should be -7 and 7 respectively. For the k-folds procedure, use 5 folds. Print the lambda and the coefficients.

In [140]:
result = find_optimal_lambda(y_ef_train, ef_train, (np.exp(-7), np.exp(7)), 5)
print("Optimal lambda:", result["optimal_lambda"])
print("Coefficients:", result["optimal_coef"])

c:\Users\Nicolas Silva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.857e+01, tolerance: 4.342e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\Nicolas Silva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.022e+01, tolerance: 3.898e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\Nicolas Silva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of i

Optimal lambda: 365.0374678653289
Coefficients: [-0. -0. -0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.
 -0. -0. -0. -0. -0. -0. -0.  0. -0. -0. -0. -0.  0.  0.  0. -0. -0. -0.
 -0. -0.  0.  0. -0.  0.  0. -0. -0. -0. -0. -0. -0.  0.  0.  0.  0.  0.
 -0. -0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0.
 -0. -0. -0.  0. -0.  0.  0. -0. -0. -0.  0.  0.  0. -0. -0. -0. -0.  0.
  0.  0.  0.  0.  0. -0.  0.  0. -0. -0.  0.  0.  0. -0.  0.  0.  0. -0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0. -0. -0. -0.  0. -0.
  0.  0. -0. -0. -0.  0.  0. -0. -0. -0. -0.  0.  0.  0.  0.  0.  0.  0.
 -0.  0.  0.  0. -0.  0.  0.  0. -0.  0.  0.  0. -0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0. -0. -0. -0. -0. -0.  0. -0.  0. -0. -0. -0. -0.
  0.  0. -0. -0. -0. -0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0. -0.
  0.  0. -0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 -0. -0. -0. -0. -0.  0. -0.  0. -0. -0. -0. -0.  0.  0. -0. -0. -0. -0.
  0

10. Now use each language's hdm package to fit a Lasso model with the theoretical optimal lambda value. You can find the documentation of the package for each language in the following links



In [141]:
#!git clone https://github.com/maxhuppertz/hdmpy.git
#!pip install multiprocess
import sys
sys.path.insert(1, "./hdmpy")
import hdmpy as hdm


In [142]:
import joblib as jbl
import multiprocess as mp
from hdmpy.help_functions import cvec, init_values
from hdmpy.LassoShooting_fit import LassoShooting_fit

In [143]:
def simul_pen(n, p, W, seed=0, fix_seed=True):
    # Check whether the seed needs to be fixed
    if fix_seed:
        # Simulate with provided seed
        g = norm.rvs(size=(n,1), random_state=seed) @ np.ones(shape=(1,p))
    else:
        # Simulate using whatever state the RNG is currently in
        g = norm.rvs(size=(n,1)) @ np.ones(shape=(1,p))

    # Calculate element of the distribution for the current draw of g
    s = n * np.amax(2 * np.abs(np.mean(W * g, axis=0)))

    # Return the result
    return s

In [144]:
################################################################################
### Python port of rlasso.R
### https://github.com/cran/hdm/blob/master/R/rlasso.R
################################################################################

################################################################################
### 1: Load modules
################################################################################

# Standard Python modules
import joblib as jbl
import multiprocess as mp
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.linear_model import LinearRegression as lm

# Other parts of hdmpy
from hdmpy.help_functions import cvec, init_values
from hdmpy.LassoShooting_fit import LassoShooting_fit

################################################################################
### 2: Define functions
################################################################################

################################################################################
### 2.1: Functions which are not in the original R package
###      These are generally helper functions to allow an implementation which
###      reads as closely to the original R code as possible, and to ease a
###      Python implementation, including parallelizing the code
################################################################################


# Define a function to simulate distributions needed for calculating X-dependent
# penalty terms
def simul_pen(n, p, W, seed=0, fix_seed=True):
    # Check whether the seed needs to be fixed
    if fix_seed:
        # Simulate with provided seed
        g = norm.rvs(size=(n,1), random_state=seed) @ np.ones(shape=(1,p))
    else:
        # Simulate using whatever state the RNG is currently in
        g = norm.rvs(size=(n,1)) @ np.ones(shape=(1,p))

    # Calculate element of the distribution for the current draw of g
    s = n * np.amax(2 * np.abs(np.mean(W * g, axis=0)))

    # Return the result
    return s

################################################################################
### 2.2: Functions which are in the original R package
################################################################################


def lambdaCalculation(homoskedastic=False, X_dependent_lambda=False,
                      lambda_start=None, c=1.1, gamma=0.1, numSim=5000, y=None,
                      x=None, par=True, corecap=np.inf, fix_seed=True):
    # Get number of observations n and number of variables p
    n, p = x.shape

    # Get number of simulations to use (if simulations are necessary)
    R = numSim

    # Go through all possible combinations of homoskedasticy/heteroskedasticity
    # and X-dependent or independent error terms. The first two cases are
    # special cases: Handling the case there homoskedastic was set to None, and
    # where lambda_start was provided.
    #
    # 1) If homoskedastic was set to None (special case)
    if homoskedastic is None:
        # Initialize lambda
        lmbda0 = lambda_start

        Ups0 = (1 / np.sqrt(n)) * np.sqrt((y**2).T @ (x**2)).T

        # Calculate the final vector of penalty terms
        lmbda = lmbda0 * Ups0

    # 2) If lambda_start was provided (special case)
    elif lambda_start is not None:
        # Check whether a homogeneous penalty term was provided (a scalar)
        if np.amax(cvec(lambda_start).shape) == 1:
            # If so, repeat that p times as the penalty term
            lmbda = np.ones(shape=(p,1)) * lambda_start
        else:
            # Otherwise, use the provided vector of penalty terms as is
            lmbda = lambda_start

    # 3) Homoskedastic and X-independent
    elif (homoskedastic == True) and (X_dependent_lambda == False):
        # Initilaize lambda
        lmbda0 = 2 * c * np.sqrt(n) * norm.ppf(1 - gamma/(2*p))

        # Use ddof=1 to be consistent with R's var() function
        Ups0 = np.sqrt(np.var(y, axis=0, ddof=1))

        # Calculate the final vector of penalty terms
        lmbda = np.zeros(shape=(p,1)) + lmbda0 * Ups0

    # 4) Homoskedastic and X-dependent
    elif (homoskedastic == True) and (X_dependent_lambda == True):
        psi = cvec((x**2).mean(axis=0))

        tXtpsi = (x.T / np.sqrt(psi)).T

        # Check whether to use parallel processing
        if par == True:
            # If so, get the number of cores to use
            cores = np.int(np.amin([mp.cpu_count(), corecap]))
        else:
            # Otherwise, use only one core (i.e. run sequentially)
            cores = 1

        # Get simulated distribution
        sim = jbl.Parallel(n_jobs=cores)(
            jbl.delayed(simul_pen)(
                n, p, tXtpsi, seed=l*20, fix_seed=fix_seed
            ) for l in np.arange(R)
        )

        # Convert it to a proper column vector
        sim = cvec(sim)

        # Initialize lambda based on the simulated quantiles
        lmbda0 = c * np.quantile(sim, q=1-gamma, axis=0)

        Ups0 = np.sqrt(np.var(y, axis=0, ddof=1))

        # Calculate the final vector of penalty terms
        lmbda = np.zeros(shape=(p,1)) + lmbda0 * Ups0

    # 5) Heteroskedastic and X-independent
    elif (homoskedastic == False) and (X_dependent_lambda == False):
        # The original includes the comment, "1=num endogenous variables"
        lmbda0 = 2 * c * np.sqrt(n) * norm.ppf(1 - gamma/(2*p*1))

        Ups0 = (1 / np.sqrt(n)) * np.sqrt((y**2).T @ (x**2)).T

        lmbda = lmbda0 * Ups0

    # 6) Heteroskedastic and X-dependent
    elif (homoskedastic == False) and (X_dependent_lambda == True):
        eh = y

        ehat = eh @ np.ones(shape=(1,p))

        xehat = x * ehat

        psi = cvec((xehat**2).mean(axis=0)).T

        tXehattpsi = (xehat / ( np.ones(shape=(n,1)) @ np.sqrt(psi) ))

        # Check whether to use parallel processing
        if par == True:
            # If so, get the number of cores to use
            cores = np.int(np.amin([mp.cpu_count(), corecap]))
        else:
            # Otherwise, use only one core (i.e. run sequentially)
            cores = 1

        # Get simulated distribution
        sim = jbl.Parallel(n_jobs=cores)(
            jbl.delayed(simul_pen)(
                n, p, tXehattpsi, seed=l*20, fix_seed=fix_seed
            ) for l in np.arange(R)
        )

        # Convert it to a proper column vector
        sim = cvec(sim)

        # Initialize lambda based on the simulated quantiles
        lmbda0 = c * np.quantile(sim, q=1-gamma, axis=0)

        Ups0 = (1 / np.sqrt(n)) * np.sqrt((y**2).T @ (x**2)).T

        # Calculate the final vector of penalty terms
        lmbda = lmbda0 * Ups0

    # Return results
    return {'lambda0': lmbda0, 'lambda': lmbda, 'Ups0': Ups0}

################################################################################
### 3: Define classes
################################################################################

class rlasso:
    # Initialize gamma to None to get gamma=.1/log(n)
    def __init__(self, x, y, colnames=None, post=True, intercept=True,
                 model=True, homoskedastic=False, X_dependent_lambda=False,
                 lambda_start=None, c=1.1, gamma=None, numSim=5000, numIter=15,
                 tol=10**(-5), threshold=-np.inf, par=True, corecap=np.inf,
                 fix_seed=True):
        # Initialize internal variables
        if isinstance(x, pd.DataFrame) and colnames is None:
            colnames = x.columns

        self.x = np.array(x).astype(np.float32)
        self.y = cvec(y).astype(np.float32)

        self.n, self.p = self.x.shape

        if colnames is None:
            self.colnames = ['V' + str(i+1) for i in np.arange(self.p)]
        else:
            self.colnames = colnames

        # Unused line in the original code
        # ind_names = np.arange(self.p) + 1

        self.post = post
        self.intercept = intercept
        self.model = model
        self.homoskedastic = homoskedastic
        self.X_dependent_lambda = X_dependent_lambda
        self.lambda_start = lambda_start
        self.c = c

        if gamma is None:
            self.gamma = .1 / np.log(self.n)
        else:
            self.gamma = gamma

        self.numSim = numSim
        self.numIter = numIter
        self.tol = tol
        self.threshold = threshold

        self.par = par
        self.corecap = corecap
        self.fix_seed = fix_seed

        if (self.post == False) and (self.c is None):
            self.c = .5

        if (
                (self.post == False) and (self.homoskedastic == False)
                and (self.X_dependent_lambda == False)
                and (self.lambda_start == None) and (self.c == 1.1)
                and (self.gamma == .1 / np.log(self.n))
        ):
            self.c = .5

        # For now, instantiate estimate as None
        self.est = None

        # Calculate robust LASSO coefficients
        if self.intercept == True:
            meanx = cvec(self.x.mean(axis=0))

            self.x = self.x - np.ones(shape=(self.n,1)) @ meanx.T

            mu = self.y.mean()

            self.y = self.y - mu
        else:
            meanx = np.zeros(shape=(self.p,1))

            mu = 0

        normx = np.sqrt(np.var(self.x, axis=1, ddof=1))

        Psi = cvec(np.mean(self.x**2, axis=0))

        ind = np.zeros(shape=(self.p,1)).astype(bool)

        XX = self.x.T @ self.x

        Xy = self.x.T @ self.y

        startingval = init_values(self.x, self.y)['residuals']

        pen = lambdaCalculation(homoskedastic=self.homoskedastic,
                                X_dependent_lambda=self.X_dependent_lambda,
                                lambda_start=self.lambda_start, c=self.c,
                                gamma=self.gamma, numSim=self.numSim,
                                y=startingval, x=self.x, par=self.par,
                                corecap=self.corecap, fix_seed=self.fix_seed)

        lmbda = pen['lambda']
        Ups0 = Ups1 = pen['Ups0']
        lmbda0 = pen['lambda0']

        mm = 1
        s0 = np.sqrt(np.var(y, axis=0, ddof=1))

        while mm <= self.numIter:
            if (mm == 1) and self.post:
                coefTemp = (
                    LassoShooting_fit(self.x, self.y, lmbda/2, XX=XX,
                                      Xy=Xy)['coefficients']
                )
            else:
                coefTemp = (
                    LassoShooting_fit(self.x, self.y, lmbda, XX=XX,
                                      Xy=Xy)['coefficients']
                )

            coefTemp[np.isnan(coefTemp)] = 0

            ind1 = (np.abs(coefTemp) > 0)

            x1 = self.x[:, ind1[:,0]]

            if x1.shape[1] == 0:
                if self.intercept:
                    intercept_value = np.mean(self.y + mu)

                    coef = np.zeros(shape=(self.p+1,1))

                    coef = (
                        pd.DataFrame(coef,
                                     index=['(Intercept)']+list(self.colnames))
                    )
                else:
                    intercept_value = np.mean(self.y)

                    coef = np.zeros(shape=(self.p,1))

                    coef = pd.DataFrame(coef, index=self.colnames)

                self.est = {
                    'coefficients': coef,
                    'beta': np.zeros(shape=(self.p,1)),
                    'intercept': intercept_value,
                    'index': pd.DataFrame(np.zeros(shape=(self.p,1)).astype(
                        bool),
                                          index=self.colnames),
                    'lambda': lmbda,
                    'lambda0': lmbda0,
                    'loadings': Ups0,
                    'residuals': self.y - np.mean(self.y),
                    'sigma': np.var(self.y, axis=0, ddof=1),
                    'iter': mm,
                    #'call': Not a Python option
                    'options': {'post': self.post, 'intercept': self.intercept,
                                'ind.scale': ind, 'mu': mu, 'meanx': meanx}
                }

                if self.model:
                    self.est['model'] = self.x
                else:
                    self.est['model'] = None

                self.est['tss'] = self.est['rss'] = (
                    ((self.y - np.mean(self.y))**2).sum()
                )

                self.est['dev']: self.y - np.mean(self.y)
                # In R, return() breaks while loops
                return

            # Refinement variance estimation
            if self.post:
                reg = lm(fit_intercept=False).fit(x1, self.y)

                coefT = reg.coef_.T

                coefT[np.isnan(coefT)] = 0

                e1 = self.y - x1 @ coefT

                coefTemp[ind1[:,0]] = coefT
            else:
                e1 = self.y - x1@ coefTemp[ind1[:,0]]

            s1 = np.sqrt(np.var(e1, ddof=1))

            # Homoskedastic and X-independent
            if (
                    (self.homoskedastic == True)
                    and (self.X_dependent_lambda == False)
            ):
                Ups1 = s1 * Psi

                lmbda = pen['lambda0'] * Ups1

            # Homoskedastic and X-dependent
            elif (
                    (self.homoskedastic == True)
                    and (self.X_dependent_lambda == True)
            ):
                Ups1 = s1 * Psi

                lmbda = pen['lambda0'] * Ups1

            # Heteroskedastic and X-independent
            elif (
                    (self.homoskedastic == False)
                    and (self.X_dependent_lambda == False)
            ):
                Ups1 = (
                    (1/np.sqrt(self.n)) * np.sqrt((e1**2).T @ self.x**2).T
                )

                lmbda = pen['lambda0'] * Ups1

            # Heteroskedastic and X-dependent
            elif (
                    (self.homoskedastic == False)
                    and (self.X_dependent_lambda == True)
            ):
                lc = lambdaCalculation(homoskedastic=self.homoskedastic,
                                       X_dependent_lambda=
                                       self.X_dependent_lambda,
                                       lambda_start=self.lambda_start,
                                       c=self.c, gamma=self.gamma,
                                       numSim=self.numSim, y=e1, x=self.x,
                                       par=self.par, corecap=self.corecap,
                                       fix_seed=self.fix_seed)

                Ups1 = lc['Ups0']

                lmbda = lc['lambda']

            # If homoskedastic is set to None
            elif self.homoskedastic is None:
                Ups1 = (
                    (1/np.sqrt(self.n)) * np.sqrt((e1**2).T @ self.x**2).T
                )

                lmbda = pen['lambda0'] * Ups1

            mm = mm + 1

            if np.abs(s0 - s1) < self.tol:
                break

            s0 = s1

        if x1.shape[1] == 0:
            #coefTemp = None
            ind1 = np.zeros(shape=(self.p,1))

        coefTemp = cvec(coefTemp)

        coefTemp[np.abs(coefTemp) < self.threshold] = 0

        coefTemp = pd.DataFrame(coefTemp, index=self.colnames)

        ind1 = cvec(ind1)

        ind1 = pd.DataFrame(ind1, index=self.colnames)

        if self.intercept:
            if mu is None:
                mu = 0
            if meanx is None:
                meanx = np.zeros(shape=(coefTemp.shape[0],1))
            if ind.sum() == 0:
                intercept_value = mu - (meanx * coefTemp).sum()
            else:
                intercept_value = mu - (meanx * coefTemp).sum()
        else:
            intercept_value = np.nan

        if self.intercept:
            beta = (
                np.concatenate([cvec(intercept_value), coefTemp.values], axis=0)
            )

            beta = pd.DataFrame(beta, index=['(Intercept)']+list(self.colnames))
        else:
            beta = coefTemp

        s1 = np.sqrt(np.var(e1, ddof=1))

        self.est = {
            'coefficients': beta,
            'beta': pd.DataFrame(coefTemp, index=self.colnames),
            'intercept': intercept_value,
            'index': ind1,
            'lambda': pd.DataFrame(lmbda, index=self.colnames),
            'lambda0': lmbda0,
            'loadings': Ups1,
            'residuals': cvec(e1),
            'sigma': s1,
            'iter': mm,
            #'call': Not a Python option
            'options': {'post': self.post, 'intercept': self.intercept,
                        'ind.scale': ind, 'mu': mu, 'meanx': meanx},
            'model': model
        }

        if model:
            self.x = self.x + np.ones(shape=(self.n,1)) @ meanx.T

            self.est['model'] = self.x
        else:
            self.est['model'] = None

        self.est['tss'] = ((self.y - np.mean(self.y))**2).sum()
        self.est['rss'] = (self.est['residuals']**2).sum()
        self.est['dev'] = self.y - np.mean(self.y)

11. Report the testing MSE and R2 for the OLS model and, the cross-validation lambda Lasso model, and the hdm theoretical lambda model. Use the function programmed in 4. to compute the predictions of the cross-validation Lasso model.

In [145]:
from scipy.stats import norm
from sklearn.metrics import mean_squared_error

#OLS
ols_results = model.fit()
residuals = ols_results.resid

ols_mse = np.mean(residuals ** 2)
print("OLS Model MSE:", ols_mse)

ols_r2 = ols_results.rsquared
print("OLS Model R^2:", ols_r2)

#Cross-Validation
optimal_lambda = result["optimal_lambda"]
y_pred_train = lasso_predict(result, ef_train)
lasso_mse_train = mean_squared_error(y_ef_train, y_pred_train)
print("Lasso Training MSE:", lasso_mse_train)

lasso_r2_train = 1 - lasso_mse_train / np.var(y_ef_train)
print("Lasso Training R^2:", lasso_r2_train)

y_pred_test = lasso_predict(result, ef_test)

lasso_mse_test = mean_squared_error(y_ef_test, y_pred_test)
print("Lasso Test MSE:", lasso_mse_test)

lasso_r2_test = 1 - lasso_mse_test / np.var(y_ef_test)
print("Lasso Test R^2:", lasso_r2_test)

# HDMPY
lasso_model = rlasso(ef_train, y_ef_train, post=True)

y_pred_test = ef_test @ lasso_model.est['beta']
y_pred_train = ef_train @ lasso_model.est['beta']

mse_train = np.mean((y_ef_train - y_pred_train) ** 2)
print("MSE Train HDMPY:", mse_train)

r2_train = 1 - mse_train / np.var(y_ef_train)
print("R^2 Train HDMPY:", r2_train)

mse_test = np.mean((y_ef_test - y_pred_test) ** 2)
print("MSE Test HDMPY:", mse_test)

r2_test = 1 - mse_test / np.var(y_ef_test)
print("R^2 Test HDMPY:", r2_test)


OLS Model MSE: 0.06592479917205021
OLS Model R^2: 0.9369958519323668
Lasso Training MSE: 1.0467051071144116
Lasso Training R^2: -0.000333173252097696
Lasso Test MSE: 0.9948105436539544
Lasso Test R^2: -4.3263961200246115e-06


c:\Users\Nicolas Silva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(
c:\Users\Nicolas Silva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(
c:\Users\Nicolas Silva\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Nicolas Silva\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\Nicolas Silva\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Nicolas Silva\

MSE Train HDMPY: 0.9310960459032546
R^2 Train HDMPY: 0.11015408650425662
MSE Test HDMPY: 0.951420069215502
R^2 Test HDMPY: 0.04361268433987198
